- `aiohttp==3.3.2`
- `tornado==5.1`
- `without uvloop`
- `Intel(R) Xeon(R) CPU E5-2697 v4`
- `Python 3.6.5 (default, Jul 18 2018, 21:30:18) [GCC 6.3.0 20170516] on linux Debian 4.9`
- `Duraion - 120 seconds, 1s warmup, 5 repeats`

In [198]:
import os
import subprocess
import pandas as pd
import time
import seaborn as sns
sns.set(style='whitegrid')
sns.set(rc={'figure.figsize':(11.7, 8.27)})

def run_experiment(exp_name: str, rps=100, duration_sec=5, handler='sleep50', silent=True) -> str:
    dump_file = f'/tmp/{exp_name}.bin'
    run_vegeta = f"echo 'GET http://localhost:8890/{handler}' | vegeta -cpus 1 attack -rate {rps} -duration {duration_sec}s -timeout 1s -name {exp_name} -workers 1 > {dump_file}"
    p = subprocess.run(run_vegeta, shell=True)
    if not silent:
        print(p)
        print('---'*10)
        print_report = f"cat {dump_file} | vegeta report -reporter text > /tmp/res.txt"
        subprocess.run(print_report, shell=True)
        print(open('/tmp/res.txt').read())

def read_experiment(csv_path: str, warmup_time_sec = 0.2) -> pd.DataFrame:
    names = ['unix_ts_ns', 'http_code', 'latency_ns', 'bytes_out', 'bytes_in', 'x', 'error', 'exp_name', 'y']
    data = pd.read_csv(csv_path, header=None, names=names)
    del data['error']
    del data['x']
    del data['y']
    del data['bytes_out']
    del data['bytes_in']
    begin_ts_ns = data.unix_ts_ns.min()
    exp_start_ts_ns = begin_ts_ns + int(warmup_time_sec * 1_000_000_000)
    data = data[data.unix_ts_ns > exp_start_ts_ns]
    data.reset_index(drop=True, inplace=True)
    data['latency_ms'] = (data['latency_ns'] / 1_000_000).round(2)
    del data['unix_ts_ns']
    del data['latency_ns']
    return data

def aggr_exp(exp_names: list) -> str:
    out_csv = f'/tmp/out.csv'
    dump_files = ','.join([f'/tmp/{exp_name}.bin' for exp_name in exp_names])
    dump_ext = f"vegeta dump -inputs {dump_files} -output {out_csv} -dumper csv"
    p = subprocess.run(dump_ext, shell=True)
    # print(p)
    return out_csv

def plot_exp_data(exp_data):
    ax = sns.boxplot(x='latency_ms', y='exp_name', data=exp_data)

def latency_table(exp_data, rps):
    a = exp_data.groupby('exp_name').quantile([.95, .98, .99])
    a = a.reset_index().pivot(index='exp_name', columns='level_1', values='latency_ms')
    a = a.round(1)
    df = a.reset_index()
    #df['exp_name'], df['rps'] = df.exp_name.str.split('_rps_').str
    df['rps'] = rps
    df['rps'] = df['rps'].astype(int)
    df.sort_values('rps', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df[['rps','exp_name', 0.95, 0.98, 0.99]]
    return df


def hand_experiment(name: str, rps=100, duration_sec=3, repeats=5, **kwargs):
    exp_datas = []
    latencies = []
    for _ in range(repeats):
        run_experiment(name, rps, duration_sec, **kwargs)
        csv_file = aggr_exp([name])
        exp_data = read_experiment(csv_file, warmup_time_sec=1)
        exp_datas.append(exp_data)
        df = latency_table(exp_data, rps)
        latencies.append(df)
    latencies = pd.concat(latencies)
    return latencies, exp_datas


repeats = 3
duration_sec = 10
rps = 1000
wait_time = 3

In [205]:
lats = []
stats = []

In [208]:
def experiment(name,
               rps, 
               lats,
               stats,
               duration_sec=duration_sec, 
               repeats=repeats,
               **kwargs
              ):
    lat, stat = hand_experiment('tornado_oldstyle', rps=rps, duration_sec=duration_sec, repeats=repeats, **kwargs)
    d = pd.concat(stat)
    d['rps'] = rps
    stats.append(stat)
    time.sleep(wait_time)
    return lat

# Tornado

In [207]:
experiment('tornado_oldstyle', 1000, lats, stats, handler='oldstyle')

level_1,rps,exp_name,0.95,0.98,0.99
0,1000,tornado_oldstyle,269.2,287.6,300.4
0,1000,tornado_oldstyle,768.3,796.7,804.9
0,1000,tornado_oldstyle,451.6,481.5,502.2


In [ ]:
experiment('tornado_async', 1000, lats, stats)

In [ ]:
experiment('tornado_async', 1200, lats, stats)

In [ ]:
experiment('tornado_async', 1400, lats, stats)

In [ ]:
experiment('tornado_hard_work', 800, lats, stats, handler='hard_work')

In [ ]:
experiment('tornado_hard_work', 1000, lats, stats, handler='hard_work')

In [ ]:
experiment('tornado_hard_work', 1100, lats, stats, handler='hard_work')

In [ ]:
experiment('tornado_hard_work', 1200, lats, stats, handler='hard_work')

In [ ]:
experiment('tornado_hard_work_oldstyle', 800, lats, stats, handler='hard_work_oldstyle')

In [ ]:
experiment('tornado_hard_work_oldstyle', 900, lats, stats, handler='hard_work_oldstyle')

In [ ]:
experiment('tornado_hard_work_oldstyle', 1000, lats, stats, handler='hard_work_oldstyle')

In [ ]:
experiment('tornado_hard_work_uvloop', 1200, lats, stats, handler='hard_work')

# aiohttp

In [ ]:
experiment('aiohttp', 1000, lats, stats)

In [ ]:
experiment('aiohttp', 1500, lats, stats)

In [ ]:
experiment('aiohttp', 2000, lats, stats)

In [ ]:
experiment('aiohttp', 2500, lats, stats)

In [ ]:
experiment('aiohttp', 3000, lats, stats)

In [ ]:
experiment('aiohttp', 800, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 1000, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 1100, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 1200, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 1800, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 2000, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 2200, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 2300, lats, stats, handler='hard_work')

In [ ]:
experiment('aiohttp', 2400, lats, stats, handler='hard_work')

In [ ]:
#experiment('aiohttp_uvloop_hard_work', 2400, lats, stats, handler='hard_work')

In [ ]:
#experiment('aiohttp_uvloop_hard_work', 2300, lats, stats, handler='hard_work')

In [ ]:
#experiment('aiohttp_uvloop_hard_work', 2000, lats, stats, handler='hard_work')

In [ ]:
#experiment('aiohttp_uvloop_hard_work', 800, lats, stats, handler='hard_work')

In [103]:
def results(lats: list) -> pd.DataFrame:
    df = pd.concat(lats)
    df = df[df[0.99] < 200]
    df = df.groupby(['exp_name', 'rps']).mean().sort_values(0.98).round(2)
    df = df.reset_index()
    renamer = {
        0.95: '95% ms',
        0.98: '98% ms',
        0.99: '99% ms',
    }
    df = df.rename(renamer, axis=1)
    return df

In [129]:
bar_color = '#e5cbb7'
bars = ['95% ms', '98% ms', '99% ms']

In [143]:
res = results(lats)
res = res[~res.exp_name.str.contains('hard_work')]
res.style.bar(subset=bars, color=bar_color)

level_1,exp_name,rps,95% ms,98% ms,99% ms


In [145]:
res = results(lats)
res = res[~res.exp_name.str.contains('hard_work') & res.exp_name.str.contains('tornado')]
res.style.bar(subset=bars, color=bar_color)

level_1,exp_name,rps,95% ms,98% ms,99% ms


In [146]:
res = results(lats)
res = res[~res.exp_name.str.contains('hard_work') & res.exp_name.str.contains('aiohttp')]
res.style.bar(subset=bars, color=bar_color)

level_1,exp_name,rps,95% ms,98% ms,99% ms


In [147]:
res = results(lats)
res = res[res.exp_name.str.contains('hard_work')]
res.style.bar(subset=bars, color=bar_color)

level_1,exp_name,rps,95% ms,98% ms,99% ms
0,tornado_hard_work_uvloop,1200,127.98,130.56,132.28
1,aiohttp_uvloop_hard_work,2500,133.6,139.3,145.46


In [148]:
# mean cpu time of process

In [149]:
res = results(lats)
res = res[res.exp_name.str.contains('hard_work') & res.exp_name.str.contains('aiohttp')]
res.style.bar(subset=bars, color=bar_color)

/home/dkhodakov/.pyenv/versions/3.6.5/envs/avio/lib/python3.6/site-packages/pandas/io/formats/style.py:960: RuntimeWarning: divide by zero encountered in double_scalars
  zero_normed = width * (0 - s.min()) / (s.max() - s.min())


level_1,exp_name,rps,95% ms,98% ms,99% ms
1,aiohttp_uvloop_hard_work,2500,133.6,139.3,145.46


In [138]:
res = results(lats)
res = res[res.exp_name.str.contains('hard_work') & res.exp_name.str.contains('tornado')]
res.style.bar(subset=bars, color=bar_color)

level_1,exp_name,rps,95% ms,98% ms,99% ms
9,tornado_hard_work,1000,132.47,133.97,135.13
12,tornado_hard_work,1200,175.47,179.93,182.7
13,tornado_hard_work_oldstyle,1000,179.27,185.5,188.97
